If Dockerfiles have not been modified, connect to the Jupyter server with ```http://localhost:8014/tree?token=predict-perception```.  

This notebook provides an interface to predict the perception of an encoding as a ```.json``` given a model as a ```.pkl``` that accepts the encoding.  

In [ ]:
target_dir = "data"
input_dir = "thomson-segmented-point-clouds-combined-encoded"
model_filename = "best-segmented-point-cloud-models/epsilonsvr_segmentedpc_safer_C1.0_gamma0.0001_kernelrbf_r20.013.pkl.pkl"
scaler_filename = "best-segmented-point-cloud-models/scaler.pkl"
output_dir = "thomson-perceptions"
perception = "safer"

In [ ]:
import joblib
import matplotlib.pyplot as plt
import numpy as np

import json
import os
from pathlib import Path

In [ ]:
with open(os.path.join(target_dir, model_filename), "rb") as fp:
    svr = joblib.load(fp)
with open(os.path.join(target_dir, scaler_filename), "rb") as fp:
    scaler = joblib.load(fp)
xs_path = []
for dirpath, dirnames, filenames in os.walk(os.path.join(target_dir, input_dir)):
    xs_path.extend(filenames)
    break
xs_id = {}
for x_path in xs_path:
    with open(os.path.join(target_dir, input_dir, x_path), 'r') as fp:
        xs_id['.'.join(x_path.split('.')[:-1])] = json.load(fp)
X = list(xs_id.values())
X_scaled = scaler.transform(X)
y = svr.predict(X_scaled)

if not Path(os.path.join(target_dir, output_dir)).is_dir():
    Path(os.path.join(target_dir, output_dir)).mkdir(parents=True, exist_ok=True)
for id, prediction in zip(xs_id.keys(), y):
    predictions = {}
    if Path(os.path.join(target_dir, output_dir, f"{id}.json")).is_file():
        with open(Path(os.path.join(target_dir, output_dir, f"{id}.json")), 'r') as fp:
            predictions = json.load(fp)
            if not type(predictions) == dict:
                predictions = {}
    if perception in predictions:
        continue
    predictions[perception] = prediction
    with open(Path(os.path.join(target_dir, output_dir, f"{id}.json")), 'w') as fp:
        json.dump(predictions, fp)